In [ ]:
import warnings 
warnings.filterwarnings('ignore')
context = create_default_context()
bq_db= context.get_ds('YOUR CONTEXT')

In [ ]:
import pandas as pd
import numpy as np
from ggv.utils.context import create_default_context
import warnings
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
context = create_default_context()
bq_db= context.get_ds('YOUR CONTEXT')

In [ ]:
query= f""" 
SELECT
  standard_order_request_id,
  booking_type,
  created_at,
  first_assigned_at,
  request_pickup_time,
  last_assigned_at,
  user_unique_id,
  status
FROM
  `TableName`
WHERE
  DATE(created_at) >=DATE('2023-01-01')
  AND product_name = 'TRANSPORT'
  AND status = 'completed'
""" 
data = bq_db.df_from_sql(query) 

In [ ]:
data['created_at'] - data['last_assigned_at']

In [ ]:
data['created_at'] = pd.to_datetime(data['created_at'], format="%Y-%m-%d %H:%M:%S")
data['created_at_date'] = pd.to_datetime(data['created_at']).dt.date
data['created_at_time'] = pd.to_datetime(data['created_at']).dt.strftime("%H:%M")
data['created_at_time'] = pd.to_datetime(data['created_at_time'], format="%H:%M")
data['created_at_time'] = pd.to_datetime(data['created_at_time']).dt.strftime("%Y-%m-%d %H:%M:%S")
data['last_assigned_at'] = pd.to_datetime(data['last_assigned_at'], format="%Y-%m-%d %H:%M:%S")
data['last_assigned_at_date'] = pd.to_datetime(data['last_assigned_at']).dt.date
data['last_assigned_at_time'] = pd.to_datetime(data['last_assigned_at']).dt.strftime("%H:%M")
data['last_assigned_at_time'] = pd.to_datetime(data['last_assigned_at_time'], format="%H:%M")
data['last_assigned_at_time'] = pd.to_datetime(data['last_assigned_at_time']).dt.strftime("%Y-%m-%d %H:%M:%S")
data['first_assigned_at'] = pd.to_datetime(data['first_assigned_at'], format="%Y-%m-%d %H:%M:%S")
data['first_assigned_at_date'] = pd.to_datetime(data['first_assigned_at']).dt.date
data['first_assigned_at_time'] = pd.to_datetime(data['first_assigned_at']).dt.strftime("%H:%M")
data['first_assigned_at_time'] = pd.to_datetime(data['first_assigned_at_time'], format="%H:%M")
data['first_assigned_at_time'] = pd.to_datetime(data['first_assigned_at_time']).dt.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
df_new = pd.DataFrame(data)
df_new['diff_time'] = data['created_at'] - data['last_assigned_at']
df_new['time_diff'] = df_new['diff_time'].dt.total_seconds() / 60

intervals = [-np.inf, 1, 2, 3, 4, 5, 10, 15, 20, 25, np.inf]
labels_1 = ['Under 1 mins', '1 min to 2', '2 min to 3', '3 min to 4', '4 min to 5', '5 mins to 10', '10 mins to 15', '15 mins to 20', '20 mins to 25', '25+ mins']
df_new['name_diff'] = pd.cut(np.abs(df_new['time_diff']), intervals, labels=labels_1)

value_counts = df_new['name_diff'].value_counts().reset_index()
value_counts.columns = ['Time Difference', 'Frequency']

fig = px.bar(value_counts, x='Frequency', y='Time Difference', orientation='h',
             title='Created at VS Last assigned at')

fig.show()

In [ ]:
df_new_2 = pd.DataFrame(data)
df_new_2['diff_time_2'] = data['created_at'] - data['first_assigned_at']
df_new_2['time_diff_2'] = df_new_2['diff_time_2'].dt.total_seconds() / 60

intervals = [-np.inf, 1, 2, 3, 4, 5, 10, 15, 20, 25, np.inf]
labels_1 = ['Under 1 mins', '1 min to 2', '2 min to 3', '3 min to 4', '4 min to 5', '5 mins to 10', '10 mins to 15', '15 mins to 20', '20 mins to 25', '25+ mins']
df_new_2['name_diff_2'] = pd.cut(np.abs(df_new_2['time_diff_2']), intervals, labels=labels_1)

value_counts_2 = df_new_2['name_diff_2'].value_counts().reset_index()
value_counts_2.columns = ['Time Difference', 'Frequency']

fig = px.bar(value_counts_2, x='Frequency', y='Time Difference', orientation='h',
             title='Created at VS First assigned at')

fig.show()

In [ ]:
df_new_3 = df_new_2[df_new_2['name_diff_2'] == 'Under 1 mins']
df_new_3['time_diff_2'] = df_new_3['time_diff_2'] * 60

intervals_2 = [-np.inf, 10, 20, 30, 40, 50, np.inf]
labels_2 = ['Under 10 sec', '10 sec to 20', '20 sec to 30', '30 sec to 40', '40 sec to 50', '50 sec to 60']
df_new_3['name_diff_2'] = pd.cut(np.abs(df_new_3['time_diff_2']), intervals_2, labels=labels_2)

value_counts_3 = df_new_3['name_diff_2'].value_counts().reset_index()
value_counts_3.columns = ['Time Difference', 'Frequency']

fig = px.bar(value_counts_3, x='Frequency', y='Time Difference', orientation='h',
             title='Created at VS First assigned at')

fig.show()

In [ ]:
df_new_4 = df_new[df_new['name_diff'] == 'Under 1 mins']
df_new_4['time_diff'] = df_new_4['time_diff'] * 60

intervals_2 = [-np.inf, 10, 20, 30, 40, 50, np.inf]
labels_2 = ['Under 10 sec', '10 sec to 20', '20 sec to 30', '30 sec to 40', '40 sec to 50', '50 sec to 60']
df_new_4['name_diff'] = pd.cut(np.abs(df_new_4['time_diff']), intervals_2, labels=labels_2)

value_counts_4 = df_new_4['name_diff'].value_counts().reset_index()
value_counts_4.columns = ['Time Difference', 'Frequency']

fig = px.bar(value_counts_4, x='Frequency', y='Time Difference', orientation='h',
             title='Created at VS Last assigned at')

fig.show()